In [ ]:
# Librerias para crear los vectores de caracteristicas
import os
import numpy as np
from PIL import Image
import pywt
import pickle
# Liberias para crear el modelo de clasificacion
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score, f1_score, precision_score,confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def preprocess_images(input_path, output_path, target_size=(256, 256)):
    os.makedirs(output_path, exist_ok=True)
    images = os.listdir(input_path)

    for image_name in images:
        image_path = os.path.join(input_path, image_name)
        imagen = Image.open(image_path)

        imagen = imagen.resize(target_size, Image.LANCZOS)

        output_image_path = os.path.join(output_path, image_name)
        imagen.save(output_image_path)

def load_labels(input_path):
    images = os.listdir(input_path)
    labels = [image_name[:3] for image_name in images]
    return labels

def extract_features(image, num_levels=1):
    LL = image.copy()
    for _ in range(num_levels):
        LL, (LH, HL, HH) = pywt.dwt2(LL, 'haar')
        LL = np.max(LL, axis=0)
        LH = np.max(LH, axis=0)
        HL = np.max(HL, axis=0)
        HH = np.max(HH, axis=0)
        LL = np.concatenate((LL, LH, HL, HH))

    return LL.flatten()

def preprocess_and_extract_features(input_path, output_path, num_levels=1):
    preprocess_images(input_path, output_path)
    images = os.listdir(output_path)
    features_list = []

    for image_name in images:
        image_path = os.path.join(output_path, image_name)
        imagen = Image.open(image_path)
        imagen = imagen.convert('L')
        imagen = np.array(imagen)
        features = extract_features(imagen, num_levels)
        features_list.append(features)

    return features_list

def normalize_features(features):
    scaler = MinMaxScaler()
    features_normalized = [scaler.fit_transform(feature.reshape(-1, 1)).flatten() for feature in features]
    return features_normalized


In [ ]:
# Base de datos
#from google.colab import files
#uploaded = files.upload()

# Cargando datos
#Nombre_Data_Set = 'iris - iris.csv'
#iris = pd.read_csv(Nombre_Data_Set)

# Directorios de entrada y salida
from google.colab import drive
drive.mount('/content/drive')

path_images = '/content/drive/MyDrive/ML-1/images/'
path_save = '/content/drive/MyDrive/ML-1/images_p/'

# Obtener etiquetas
labels = load_labels(path_images)

# Obtener características independientes
indep = preprocess_and_extract_features(path_images, path_save, num_levels=1)


# Normalizar las características independientes
indep_normalized = normalize_features(indep)

# Combinar etiquetas y características independientes en un diccionario
dataset = {'labels': labels, 'indep': indep_normalized}

# Nombre del archivo para guardar el dataset
output_file = 'dataset.pkl'

# Guardar el dataset en un archivo utilizando pickle
with open(output_file, 'wb') as file:
    pickle.dump(dataset, file)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open('dataset.pkl', 'rb') as file:
    dataset = pickle.load(file)
index = np.arange(len(dataset['labels']))
rnd = np.random.RandomState(123)
suffle_index = rnd.permutation(index)

x_shuffle, y_shuffle = [], []
for i in range(len(suffle_index)):
    x_shuffle.append(dataset['indep'][suffle_index[i]])
    y_shuffle.append(dataset['labels'][suffle_index[i]])
x_shuffle = np.array(x_shuffle)
y_shuffle = np.array(y_shuffle)



In [35]:
from sklearn import svm
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd


svm_model = svm.SVC(kernel='linear', decision_function_shape='ovr')

for i in range(3,11):
  kf = StratifiedKFold(n_splits=i, shuffle=True, random_state=42)

  precision = []
  recall = []
  f1 = []
  c_matrix = []

  for t, v in kf.split(x_shuffle, y_shuffle):
      x_trainf = x_shuffle[t]
      y_trainf = y_shuffle[t]
      x_valf = x_shuffle[v]
      y_valf = y_shuffle[v]

      svm_model.fit(x_trainf, y_trainf)

      y_pred = svm_model.predict(x_valf)

      p = precision_score(y_valf, y_pred, average='weighted')
      r = recall_score(y_valf, y_pred, average='weighted')
      f = f1_score(y_valf, y_pred, average='weighted')
      confusion = confusion_matrix(y_valf, y_pred)

      precision.append(p)
      recall.append(r)
      f1.append(f)
      c_matrix.append(confusion)


  print("k: "+str(i))
  results = pd.DataFrame({
      'Precision': precision,
      'Recall': recall,
      'F1-Score': f1
  })

  print(results)


k: 3
   Precision    Recall  F1-Score
0   0.436776  0.431655  0.423478
1   0.445147  0.451264  0.443075
2   0.464749  0.469314  0.461984
k: 4
   Precision    Recall  F1-Score
0   0.456517  0.447115  0.434810
1   0.446792  0.447115  0.437022
2   0.460683  0.447115  0.447195
3   0.446527  0.456731  0.446270
k: 5
   Precision    Recall  F1-Score
0   0.479495  0.461078  0.455851
1   0.429351  0.431138  0.417441
2   0.491232  0.475904  0.471771
3   0.472415  0.445783  0.443873
4   0.466658  0.475904  0.466529
k: 6
   Precision    Recall  F1-Score
0   0.471552  0.453237  0.449194
1   0.473435  0.467626  0.459366
2   0.497886  0.489209  0.481690
3   0.452782  0.446043  0.442474
4   0.493280  0.478261  0.472325
5   0.485285  0.485507  0.480457
k: 7
   Precision    Recall  F1-Score
0   0.443372  0.428571  0.423779
1   0.520572  0.495798  0.495719
2   0.498923  0.504202  0.489577
3   0.486160  0.478992  0.467353
4   0.469755  0.470588  0.463919
5   0.425278  0.420168  0.408664
6   0.485102  0.48